# Explanation:
Given SHOCK get to VALUE by only changing one letter at a time.

This problem can be formatted as a shortest path problem in a graph, where each word is a node and nodes are connected by an edge if their edit distance is exactly 1.

In [27]:
import itertools as itt
from collections import defaultdict
import requests
import random
import networkx as nx

In [2]:
page = requests.get("http://www.bestwordlist.com/4letterwords.txt").text.strip().split("\n")
page.pop(0).strip()

'There are 5454 four-letter words, https://www.bestwordlist.com/4letterwords.htm'

In [44]:
words = tuple(w.lower() for w in itt.chain.from_iterable(line.strip().split() for line in page))

In [29]:
%%time

g = nx.Graph()

for i, (a, b) in enumerate(itt.combinations(words, 2), start=1):
    if i % 3000000 == 0:
        print(i)
    
    if 1 == sum(c1 != c2 for c1, c2 in zip(a, b)):
        g.add_edge(a, b)

0
3000000
6000000
9000000
12000000
CPU times: user 32.4 s, sys: 67.1 ms, total: 32.5 s
Wall time: 32.5 s


Making a puzzle

In [66]:
def make_puzzle(graph, levels, seed=None):
    if seed is None:
        seed = random.choice(graph.nodes())
        
    if levels == 0:
        return [seed]
    
    return [seed] + make_puzzle(graph, levels - 1, random.choice(graph.neighbors(seed)))

    
random_walk(g, levels=5)

['molt', 'milt', 'melt', 'pelt', 'pell', 'well']

Solving a puzzle

In [46]:
list(nx.all_shortest_paths(g, 'kiss', 'ruts'))

[['kiss', 'kits', 'rits', 'ruts']]

# Shortest Path with Neo4J

In [7]:
from py2neo import Graph
graph = Graph()

Before begginning, delete all Word and Hop entries from graph

In [8]:
graph.cypher.execute("MATCH (n:Word) DETACH DELETE n")
graph.cypher.execute("MATCH (u:Word)-[r:HOP]-(v:WORD) DETACH DELETE r")

Add all Words to graph and index them

In [9]:
with graph.cypher.begin() as cy:
    for word in words:
        cy.append("CREATE (:Word {value: {word}})", word=word)

graph.cypher.execute("CREATE INDEX ON :Word(value)")

Add all relationships to graph

In [10]:
q = "MATCH (u:Word {value:'«r1»'}), (v:Word {value:'«r2»'}) CREATE (u)-[c:HOP]->(v)"
c = 0
for i in range(len(words)):
    for j in word_network[i]:
        c += 1
        if c % 5000 == 0:
            print(c)
        graph.cypher.execute(q, r1=words[i], r2=words[j])

5000
10000
15000
20000
25000
30000
35000


Compute shortest path(s)

In [11]:
graph.cypher.execute("""MATCH (u:Word {value:"cake"}), (v:Word {value:"pint"}), p = shortestPath((u)-[*..10]-(v)) RETURN p""")

   | p                                                                                                                                                 
---+----------------------------------------------------------------------------------------------------------------------------------------------------
 1 | (:Word {value:"cake"})-[:HOP]->(:Word {value:"cane"})-[:HOP]->(:Word {value:"pane"})-[:HOP]->(:Word {value:"pine"})-[:HOP]->(:Word {value:"pint"})